In [1]:
import torch
import os
from dotenv import load_dotenv
from langchain.schema.runnable import Runnable,RunnableLambda,RunnableSequence,RunnablePassthrough,RunnableParallel
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import AIMessagePromptTemplate, SystemMessagePromptTemplate,FewShotChatMessagePromptTemplate
from langchain.chains import LLMChain,SimpleSequentialChain
from langchain.schema.output_parser import StrOutputParser
# Initialize the language model

In [2]:
load_dotenv()

# Access them
endpoint = os.getenv("LANGCHAIN_ENDPOINT")
api_key = os.getenv("LANGCHAIN_API_KEY")
open_ai_api_key=os.getenv("open_ai_api_key")
print("LangChain Endpoint:", endpoint)
print("API Key:", api_key)

LangChain Endpoint: https://api.smith.langchain.com
API Key: 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


# RunnableLambda

In [3]:
uppercase_runnable = RunnableLambda(lambda x: x.upper())
result = uppercase_runnable.invoke("langchain")

In [4]:
def triple(x):
    return x**3
runnable=RunnableLambda(triple)
print(runnable.invoke(3))

27


In [5]:
print(runnable.batch([2,3,4]))

[8, 27, 64]


In [6]:
runnable=RunnableLambda(lambda x:x**3)
print(runnable.invoke(3))
print(runnable.batch([2,3,4]))

27
[8, 27, 64]


In [7]:
def greet(name):
    return f"hello,{name}"
runnable=RunnableLambda(greet)
print(runnable.invoke("kiran"))

hello,kiran


In [8]:
def greet(name):
    return f"hello,{name}"
runnable=RunnableLambda(lambda x:greet(x))
print(runnable.invoke("kiran"))

hello,kiran


# RunnableSequence

In [9]:
def to_lowercase(text):
    return text.lower()
def replace_spaces(text):
    return text.replace(" ","-")
def summarze_text(text):
    word_count=len(text.split())
    return f"This sentence contains {word_count} words: {text}"
def append_note(text):
    return f"{text}. This message was processed successfully"


lower_case_runnable=RunnableLambda(lambda x:to_lowercase(x))
replace_spaces_runnable=RunnableLambda(lambda x:replace_spaces(x))
summarze_text_runnable=RunnableLambda(lambda x:summarze_text(x))
append_note_runnable=RunnableLambda(lambda x:append_note(x))

chain=RunnableSequence(
    first=summarze_text_runnable,
    middle=[replace_spaces_runnable,lower_case_runnable],
    last=append_note_runnable

)

input_data="Langchain is great"
result=chain.invoke(input_data)
print(result)

this-sentence-contains-3-words:-langchain-is-great. This message was processed successfully


In [10]:
chain=summarze_text_runnable|replace_spaces_runnable|lower_case_runnable|append_note_runnable
chain.invoke(input_data)

'this-sentence-contains-3-words:-langchain-is-great. This message was processed successfully'

In [11]:
def to_lowercase(text):
    return text.lower()
def replace_spaces(text):
    return text.replace(" ","-")

lower_case_runnable=RunnableLambda(lambda x:to_lowercase(x))
replace_spaces_runnable=RunnableLambda(lambda x:replace_spaces(x))
chain=RunnableSequence(replace_spaces_runnable,lower_case_runnable)

prompt=PromptTemplate.from_template("Generate a short description for :{term}?")
llm=ChatOpenAI(temperature=0.7,model_name="gpt-4-turbo",api_key=open_ai_api_key)
full_chain=chain|prompt|llm|StrOutputParser()

input_data="Langchain is great"
result=full_chain.invoke(input_data)
print(result)

:langchain-is-great? appears to be a tag or label possibly used in a digital context, such as social media, programming, or categorization within a database. Although the specific origins or usage of this tag are not detailed, it likely serves to express enthusiasm or positive sentiment about "LangChain," which could refer to a technology, framework, or concept related to language processing or chain structures in computing. This tag helps in grouping discussions, content, or entries that share a positive view or successful experiences with LangChain.


#  RunnablePassthrough

In [12]:

prompt=PromptTemplate.from_template("What is 20 multiply by :{num}?")
runnable_chain={"num":RunnablePassthrough()}|prompt|llm|StrOutputParser()
runnable_chain.invoke(20)

'The product of 20 multiplied by 20 is 400.'

In [13]:
from datetime import datetime
def get_today(day):
    return datetime.today().strftime("%B %d")
prompt =PromptTemplate.from_template(
    "List {n} significant historical events that occured on {today}.Include a brief description of each"
)
chain={"today":RunnableLambda(get_today),"n":RunnablePassthrough()}|prompt| llm | StrOutputParser()
print(chain.invoke(2))

1. **World Trade Center Bombing (1993)** - On February 26, 1993, a terrorist bomb exploded in a garage of the North Tower of the World Trade Center in New York City. This attack killed six people and injured over a thousand. The bombing was executed with a rented van filled with about 1,200 pounds of explosives. This event marked a significant moment in American history as it was one of the earliest acts of organized terrorism targeting significant U.S. landmarks, leading to increased awareness and changes in security measures across the country.

2. **Grand Canyon Becomes a National Park (1919)** - On February 26, 1919, the Grand Canyon was designated as a National Park by President Woodrow Wilson. The Grand Canyon, known for its vast size and intricate and colorful landscape, had been a national monument for 11 years prior to becoming the 17th U.S. National Park. This designation helped preserve its natural beauty and biodiversity, allowing for managed public access and protection un

In [15]:
def double(x):
    return x**2
def add(x):
    return x+2
def squared(x):
    return x**2
runnable_1=RunnableLambda(double)
runnable_2=RunnableLambda(add)
runnable_3=RunnableLambda(squared)

sequence=runnable_1|RunnableParallel({"addition":runnable_2,"squared":runnable_3})
print(sequence.invoke(2))

{'addition': 6, 'squared': 16}


In [16]:
print(sequence.batch([3,4]))

[{'addition': 11, 'squared': 81}, {'addition': 18, 'squared': 256}]
